In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def lidar_to_2d_front_view(points,
                           v_res,
                           h_res,
                           v_fov,
                           val,
                           cmap="jet",
                           saveto=None,
                           y_fudge=0.0
                           ):
    

    # DUMMY PROOFING
    # assert len(v_fov) ==2, "v_fov must be list/tuple of length 2"
    # assert v_fov[0] <= 0, "first element in v_fov must be 0 or negative"
    # assert val in {"depth", "height", "reflectance"}
        # 'val must be one of {"depth", "height", "reflectance"}'


    x_lidar = points[:, 0]
    y_lidar = points[:, 1]
    z_lidar = points[:, 2]
    r_lidar = points[:, 3] # Reflectance
    
    l_lidar = points[:, 4] #label
    unique_values = np.unique(l_lidar)

    # Print the unique values
    print("Unique values of l_lidar:")
    print(unique_values)
    
    # Distance relative to origin when looked from top
    d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2)
    # Absolute distance relative to origin
    # d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2, z_lidar ** 2)

    v_fov_total = -v_fov[0] + v_fov[1]

    # Convert to Radians
    v_res_rad = v_res * (np.pi/180)
    h_res_rad = h_res * (np.pi/180)

    # PROJECT INTO IMAGE COORDINATES
    x_img = np.arctan2(-y_lidar, x_lidar)/ h_res_rad
    y_img = np.arctan2(z_lidar, d_lidar)/ v_res_rad
    

    
    learning_map = {
    0: 0,     # "unlabeled"
    1: 0,     # "outlier" mapped to "unlabeled"
    10: 1,    # "car"
    11: 2,    # "bicycle"
    13: 5,    # "bus" mapped to "other-vehicle"
    15: 3,    # "motorcycle"
    16: 5,    # "on-rails" mapped to "other-vehicle"
    18: 4,    # "truck"
    20: 5,    # "other-vehicle"
    30: 6,    # "person"
    31: 7,    # "bicyclist"
    32: 8,    # "motorcyclist"
    40: 9,    # "road"
    44: 10,   # "parking"
    48: 11,   # "sidewalk"
    49: 12,   # "other-ground"
    50: 13,   # "building"
    51: 14,   # "fence"
    52: 0,    # "other-structure" mapped to "unlabeled"
    60: 9,    # "lane-marking" to "road"
    70: 15,   # "vegetation"
    71: 16,   # "trunk"
    72: 17,   # "terrain"
    80: 18,   # "pole"
    81: 19,   # "traffic-sign"
    99: 0,    # "other-object" to "unlabeled"
    252: 1,   # "moving-car" to "car"
    253: 7,   # "moving-bicyclist" to "bicyclist"
    254: 6,   # "moving-person" to "person"
    255: 8,   # "moving-motorcyclist" to "motorcyclist"
    256: 5,   # "moving-on-rails" mapped to "other-vehicle"
    257: 5,   # "moving-bus" mapped to "other-vehicle"
    258: 4,   # "moving-truck" to "truck"
    259: 5,    # "moving-other-vehicle" to "other-vehicle"
 
}


    
    # SHIFT COORDINATES TO MAKE 0,0 THE MINIMUM
    x_min = -360.0 / h_res / 2  # Theoretical min x value based on sensor specs
    x_img -= x_min              # Shift
    x_max = 360.0 / h_res       # Theoretical max x value after shifting
    #print("x_img",x_img)
    #print("x_max",x_max)

    y_min = v_fov[0] / v_res    # theoretical min y value based on sensor specs
    y_img -= y_min              # Shift
    y_max = v_fov_total / v_res # Theoretical max x value after shifting

    y_max += y_fudge            # Fudge factor if the calculations based on
                                # spec sheet do not match the range of
                                # angles collected by in the data.
                                
    pixel_values1=[] # save the colormap for each label value in list
    
    # WHAT DATA TO USE TO ENCODE THE VALUE FOR EACH PIXEL
    if val == "intensity":
        pixel_values = r_lidar
    elif val == "height":
        pixel_values = z_lidar
    elif val == 'label':
        pixel_values = np.array([learning_map[label] if label in learning_map else 0 for label in l_lidar])
    
    else:
        pixel_values = d_lidar
        

    #pixel_values=np.array(pixel_values1)

    # Rescale x_img and y_img to fit within 64x1024 grid
    x_img_scaled = (x_img / x_max) * 1024
    y_img_scaled = (y_img / y_max) * 64


    # Create a 2D array to store pixel values
    image_array = np.zeros((64, 1024))

    # Assign pixel values to the corresponding grid points
    for i in range(len(x_img_scaled)):
        x_index = int(round(x_img_scaled[i]))
        y_index = int(round(y_img_scaled[i]))
        if 0 <= x_index < 1024 and 0 <= y_index < 64:
            image_array[63-y_index, x_index] = pixel_values[i]

    if saveto is not None:
        np.save(saveto, image_array)

In [3]:
# set input and output folder paths
folder = 'Val'
input_folder = rf"/DATA2/Vivek/Data/Csv/csv files/Subset_Downstream_Task/{folder}"
output_folder = rf"/DATA2/Vivek/Data/Downstream Task/{folder}/label_test_lidar"
label_folder_base = r"/DATA2/Vivek/Data/data_odometry_labels/dataset/sequences"
# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [12]:
#For folder Hierarchy  with sequence folders
HRES = 0.35         # horizontal resolution (assuming 20Hz setting)
VRES=0.4
VFOV = (-25, 2) # Field of view (-ve, +ve) along vertical axis
Y_FUDGE = 0         # y fudge factor for velodyne HDL 64E
# val must be one of {"depth", "height", "intensity"


# loop through all CSV files in input folder
value='label'
# create output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # read CSV file into pandas dataframe
        csv_file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(csv_file_path)
        #df.insert(4, 'label', np.nan)
        # Extract the sequence number from the file name
        sequence = filename.split('_')[0]
        
        label_folder = os.path.join(label_folder_base, sequence, 'labels')
        
        # Remove the sequence number prefix (e.g., "03_") from CSV filename
        label_filename = filename[len(sequence)+1:].replace('.csv', '.label')
        label_file_path = os.path.join(label_folder, label_filename)
        print(label_file_path)
        if os.path.exists(label_file_path):
            label = np.fromfile(label_file_path, dtype=np.int32)
            label = label.reshape(-1)
            label = label & 0x0000FFFF

            unique_values = np.unique(label)
            
            # Print the unique values
            print("Unique values of the semantic_label:", unique_values)
            
            df['label'] = label

        # Convert DataFrame to NumPy array
        lidar = df[['x', 'y', 'z', 'intensity', 'label']].to_numpy()
        
        # apply lidar_to_2d_front_view function
        output_filename = filename.replace('.csv', '.npy')
        output_path = os.path.join(output_folder, output_filename)
        lidar_to_2d_front_view(lidar, v_res=VRES, h_res=HRES, v_fov=VFOV,
                            val=value, saveto=output_path,
                            y_fudge=Y_FUDGE)

/DATA2/Vivek/Data/data_odometry_labels/dataset/sequences/03/labels/000668.label
Unique values of the semantic_label: [ 0  1 11 40 48 50 51 70 71 72 80 81 99]
Unique values of l_lidar:
[ 0.  1. 11. 40. 48. 50. 51. 70. 71. 72. 80. 81. 99.]
/DATA2/Vivek/Data/data_odometry_labels/dataset/sequences/05/labels/001209.label
Unique values of the semantic_label: [ 0  1 10 18 40 48 49 50 51 52 70 71 72 80 81 99]
Unique values of l_lidar:
[ 0.  1. 10. 18. 40. 48. 49. 50. 51. 52. 70. 71. 72. 80. 81. 99.]
/DATA2/Vivek/Data/data_odometry_labels/dataset/sequences/06/labels/000829.label
Unique values of the semantic_label: [ 0  1 10 11 15 18 20 30 40 44 48 50 51 70 71 72 80 81 99]
Unique values of l_lidar:
[ 0.  1. 10. 11. 15. 18. 20. 30. 40. 44. 48. 50. 51. 70. 71. 72. 80. 81.
 99.]
/DATA2/Vivek/Data/data_odometry_labels/dataset/sequences/08/labels/002765.label
Unique values of the semantic_label: [  0   1  10  11  15  40  44  48  49  50  51  52  70  71  72  80  81  99
 254]
Unique values of l_lidar:


ValueError: Length of values (123594) does not match length of index (14285)